In [ ]:
fa = dict {
 'sigma': bảng chữ cái: list string,
 'states': tập trạng thái: list string,
 'start': trạng thái bắt đầu: string,
 'final': tập trạng thái kết thúc: list string,
 'delta': bảng chuyển trạng thái: dict {string : dict {string : string or list string}}
 }

In [19]:
def validate_fa(fa):
    status = True
    if 'sigma' not in fa:
        print('No sigma')
        status = False
    if 'states' not in fa:
        print('No states')
        status = False
    if 'start' not in fa:
        print('No start')
        status = False
    if fa['start'] not in fa['states']:
        print(f"Start state: {fa['start']} is not in states")
        status = False
    if 'final' not in fa:
        print('No final')
        status = False
    for state in fa['final']:
        if state not in fa['states']:
            print(f'Final state {state} is not in states')
            status = False
    if 'delta' not in fa:
        print('No delta')
        status = False
    for state in fa['delta']:
        if state not in fa['states']:
            print(f'State {state} is not in states')
            status = False
        for symbol in fa['delta'][state]:
            if symbol not in fa['sigma']:
                print(f'State {state}: Symbol {symbol} is not in sigma')
                status = False
            if type(fa['delta'][state][symbol]) == list or type(fa['delta'][state][symbol]) == tuple or type(fa['delta'][state][symbol]) == set:
                for next_state in fa['delta'][state][symbol]:
                    if next_state not in fa['states']:
                        print(f'State {state}: Next state {next_state} is not in states')
                        status = False
            else:
                if fa['delta'][state][symbol] not in fa['states']:
                    print(f'State {state}: Next state {fa["delta"][state][symbol]} is not in states')
                    status = False
    for state in fa['states']:
        if state not in fa['delta']:
            print(f'State {state} is not in delta')
            status = False
    return status

In [20]:
import graphviz
from IPython.display import display_png

def draw(fa):
    if not validate_fa(fa):
        return None
    dot = graphviz.Digraph()
    dot.attr(rankdir='LR')
    dot.attr('node', shape='circle')
    dot.node('start', shape='point')
    dot.edge('start', fa['start'])
    for state in fa['final']:
        dot.node(state, shape='doublecircle')
    for state in fa['delta']:
        for state2 in fa['delta']:
            label = ''
            for symbol in fa['delta'][state]:
                if type(fa['delta'][state][symbol]) == list:
                    if state2 in fa['delta'][state][symbol]:
                        label += symbol + ','
                else:
                    if state2 == fa['delta'][state][symbol]:
                        label += symbol + ','
            if label != '':
                label = label[:-1]
                dot.edge(state, state2, label=label)
    display_png(dot)


In [21]:
import copy

def nfa2dfa(nfa):
    if not validate_fa(nfa):
        return None
    dfa = copy.deepcopy(nfa)
    # Bỏ epsilon ra khỏi bảng chữ cái nếu có
    if 'ϵ' in dfa['sigma']:
        dfa['sigma'].remove('ϵ')
    # Tìm tập đóng epsilon
    e_closure = []
    
    for state in dfa['delta']:
        tmp = [state]
        if 'ϵ' in dfa['delta'][state]:
            i = 0
            while i < len(tmp):
                if 'ϵ' in dfa['delta'][tmp[i]]:
                    for next_state in dfa['delta'][tmp[i]]['ϵ']:
                        if next_state not in tmp:
                            tmp.append(next_state)
                i += 1
        tmp.sort()
        e_closure.append(tmp)
        if state == dfa['start']:
            dfa['start'] = ''.join(tmp)

    e_closure = sorted(sorted(e_closure, key=lambda x: ''.join(x)), key=lambda x: len(x))
    
    # Tìm tập các trạng thái mà 1 chữ cái dẫn ra từ 1 trạng thái
    while len(e_closure) > 0:
        tmp_state = e_closure.pop(0)
        name = ''.join(tmp_state)
        if name in dfa['delta']:
            for symbol in dfa['delta'][name]:
                if type(dfa['delta'][name][symbol]) == list:
                    e_closure.append(dfa['delta'][name][symbol])
        else:
            dfa['delta'][name] = {}
            for symbol in dfa['sigma']:
                dfa['delta'][name][symbol] = []
                for state in tmp_state:
                    if symbol in dfa['delta'][state]:
                        if type(dfa['delta'][state][symbol]) == list:
                            dfa['delta'][name][symbol] += dfa['delta'][state][symbol]
                        else:
                            dfa['delta'][name][symbol].append(dfa['delta'][state][symbol])
                dfa['delta'][name][symbol] = list(set(dfa['delta'][name][symbol]))
                dfa['delta'][name][symbol].sort()
                if ''.join(dfa['delta'][name][symbol]) not in dfa['delta']:
                    e_closure.append(dfa['delta'][name][symbol])
        dfa['delta'][name]['ϵ'] = tmp_state # Lưu lại đểsau này kiểm tra tập này có đỉnh kết không
    
    # Gộp các trạng thái được dẫn đến thành trạng thái mới
    old_end_states = dfa['final']
    dfa['final'] = []
    for state in dfa['delta']:
        if 'ϵ' in dfa['delta'][state]:
            for old_end_state in old_end_states:
                if old_end_state in dfa['delta'][state]['ϵ']:
                    dfa['final'].append(state) # Kiểm tra xem có phải trạng thái kết không
            del dfa['delta'][state]['ϵ'] # Bỏ epsilon
        for symbol in dfa['delta'][state]:
            if type(dfa['delta'][state][symbol]) == list:
                dfa['delta'][state][symbol] = ''.join(dfa['delta'][state][symbol])
    # Tìm các trạng thái ôtômat sử dụng
    dfa['states'] = [dfa['start']]
    queue = [dfa['start']]
    while len(queue) > 0:
        state = queue.pop(0)
        for symbol in dfa['sigma']:
            if symbol not in dfa['delta'][state]: # Đầy đủ hóa ôtômat
                dfa['delta'][state][symbol] = 'null'
                if 'null' not in dfa['delta']:
                    dfa['delta']['null'] = {}
                    dfa['states'].append('null')
                    for s in dfa['sigma']:
                        dfa['delta']['null'][s] = 'null'
            if dfa['delta'][state][symbol] not in dfa['states']: # thêm các trạng thái được dẫn tới
                dfa['states'].append(dfa['delta'][state][symbol])
                queue.append(dfa['delta'][state][symbol])
    # Bỏ các trạng thái không sử dụng
    for state in dfa['delta'].copy():
        if state not in dfa['states']:
            del dfa['delta'][state]
    return dfa

In [ ]:
dfa = {
    'sigma': bảng chữ cái: list string,
    'states': tập trạng thái: list string,
    'start': trạng thái bắt đầu: string,
    'final': tập trạng thái kết thúc: list string,
    'delta': bảng chuyển trạng thái: dict {string : dict {string : string}}
}

In [ ]:
nfa1 = {
    'sigma': ['0', '1', 'ϵ'],
    'states': ['q0', 'q1', 'q2'],
    'start': 'q0',
    'final': ['q2'],
    'delta': {
        'q0': {
            '0': ['q0', 'q1'],
            '1': ['q0'],
        },
        'q1': {
            '1': ['q2'],
        },
        'q2': {
            '0': ['q2'],
            '1': ['q2'],
            'ϵ': ['q0']
        }
    }
}
draw(nfa1)

In [ ]:
nfa2dfa(nfa1)

In [ ]:
{'sigma': ['0', '1'],
'states': ['q0', 'q0q1', 'q0q2', 'q0q1q2'],
'start': 'q0',
'final': ['q0q2', 'q0q1q2'],
'delta': {'q0': {'0': 'q0q1', '1': 'q0'},
'q0q2': {'0': 'q0q1q2', '1': 'q0q2'},
'q0q1': {'0': 'q0q1', '1': 'q0q2'},
'q0q1q2': {'0': 'q0q1q2', '1': 'q0q2'}}}

In [ ]:
draw(nfa2dfa(nfa1))

In [ ]:
nfa2 = {
    'sigma': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
    'states': ['q0', 'q1', 'q2'],
    'start': 'q0',
    'final': ['q2'],
    'delta': {
        'q0': {
            '0': ['q2'],
            '1': 'q1',
            '2': 'q1',
            '3': 'q1',
            '4': ['q1'],
            '5': ['q1', 'q2'],
            '6': 'q1',
            '7': 'q1',
            '8': 'q1',
            '9': 'q1',
        },
        'q1': {
            '0': ['q1', 'q2'],
            '1': 'q1',
                '2': 'q1',
            '3': 'q1',
            '4': 'q1',
            '5': ['q1', 'q2'],
            '6': 'q1',
            '7': 'q1',
            '8': 'q1',
            '9': 'q1',
            },
            'q2': {
    
        }
    }
}
draw(nfa2)

In [ ]:
nfa2dfa(nfa2)

In [ ]:
{'sigma': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
'states': ['q0', 'q2', 'q1', 'q1q2', 'null'],
'start': 'q0',
'final': ['q2', 'q1q2'],
'delta': {'q0': {'0': 'q2',
'1': 'q1',
'2': 'q1',
'3': 'q1',
'4': 'q1',
'5': 'q1q2',
'6': 'q1',
'7': 'q1',
'8': 'q1',
'9': 'q1'},
'q1': {'0': 'q1q2',
'1': 'q1',
'2': 'q1',
'3': 'q1',
'4': 'q1',
'5': 'q1q2',
'6': 'q1',
'7': 'q1',
'8': 'q1',
'9': 'q1'},
'q2': {'0': 'null',
'1': 'null',
'2': 'null',
'3': 'null',
'4': 'null',
'5': 'null',
'6': 'null',
'7': 'null',
'8': 'null',
'9': 'null'},
'q1q2': {'0': 'q1q2',
'1': 'q1',
'2': 'q1',
'3': 'q1',
'4': 'q1',
'5': 'q1q2',
'6': 'q1',
'7': 'q1',
'8': 'q1',
'9': 'q1'},
'null': {'0': 'null',
'1': 'null',
'2': 'null',
'3': 'null',
'4': 'null',
'5': 'null',
'6': 'null',
'7': 'null',
'8': 'null',
'9': 'null'}}}

In [ ]:
 draw(nfa2dfa(nfa2))

In [ ]:
def minimize(fa):
    # Đơn định hóa và làm đầy ôtômat + loại bỏ trạng thái thừa
    dfa = nfa2dfa(fa)
    
    # tạo bảng
    table_next = []
    for i in dfa['states']:
        for j in dfa['states']:
            table_next.append((i, j))
    
    # loại các cặp không gộp được
    def remove(e):
        col, row = e
        col_is_final = col in dfa['final']
        row_is_final = row in dfa['final']
        # loại bỏ cặp (đánh x) trong đócó 1 trạng thái kết thúc
        if col_is_final ^ row_is_final:
            return True
        
        col_transitions = dfa['delta'][col]
        row_transitions = dfa['delta'][row]

        # loại bỏ cặp (đánh x) 2 trạng thái cùng 1 chữ cái dẫn ra 2 trạng thái đãđược đánh dấu
        for symbol, col_next_state in col_transitions.items():
            row_next_state = row_transitions[symbol]
            if (col_next_state, row_next_state) in table_current:
                continue
            else:
                return True
        for symbol, row_next_state in row_transitions.items():
            col_next_state = col_transitions[symbol]
            if (col_next_state, row_next_state) in table_current:
                continue
            else:
                return True
        return False
    table_current = None
    while table_current != table_next:
        table_current = table_next
        table_next = copy.deepcopy(table_current)
        
        for e in table_current:
            if remove(e):
                table_next.remove(e)
    # gộp các trạng thái
    group = {}
    for (s1, s2) in table_next:
        group.setdefault(s1,frozenset())
        group[s1] = group[s1].union({s2})
    group = set(group.values())
    group = dict((".".join(sorted(list(g))), g) for g in group)

    def get_group(state):
        for key, val in group.items():
            if state in val:
                return key

    # tạo hàm chuyển
    new_transition_function = {}
    for state in dfa['delta']:
        new_state = get_group(state)
        new_transition_function[new_state]={}
        for symbol, next_state in dfa['delta'][state].items():
            new_next_state = get_group(next_state)
            new_transition_function[new_state][symbol] = new_next_state

    # cập nhật các thông tin
    dfa['states'] = set(group.keys())
    dfa['delta'] = new_transition_function
    dfa['final'] = set(get_group(state) for state in dfa['final'])
    dfa['start'] = get_group(dfa['start'])

    return dfa

In [ ]:
minimize(nfa1)

In [ ]:
{'sigma': ['0', '1'],
'states': {'q0', 'q0q1', 'q0q1q2.q0q2'},
'start': 'q0',
'final': {'q0q1q2.q0q2'},
'delta': {'q0': {'0': 'q0q1', '1': 'q0'},
'q0q1q2.q0q2': {'0': 'q0q1q2.q0q2', '1': 'q0q1q2.q0q2'},
'q0q1': {'0': 'q0q1', '1': 'q0q1q2.q0q2'}}}

In [ ]:
draw(minimize(nfa1))

In [ ]:
minimize(nfa2)

In [ ]:
{'sigma': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
'states': {'null', 'q0', 'q1', 'q1q2', 'q2'},
'start': 'q0',
'final': {'q1q2', 'q2'},
'delta': {'q0': {'0': 'q2',
'1': 'q1',
'2': 'q1',
'3': 'q1',
'4': 'q1',
'5': 'q1q2',
'6': 'q1',
'7': 'q1',
'8': 'q1',
'9': 'q1'},
'q1': {'0': 'q1q2',
'1': 'q1',
'2': 'q1',
'3': 'q1',
'4': 'q1',
'5': 'q1q2',
'6': 'q1',
'7': 'q1',
'8': 'q1',
'9': 'q1'},
'q2': {'0': 'null',
'1': 'null',
'2': 'null',
'3': 'null',
'4': 'null',
'5': 'null',
'6': 'null',
'7': 'null',
'8': 'null',
'9': 'null'},
'q1q2': {'0': 'q1q2',
'1': 'q1',
'2': 'q1',
'3': 'q1',
'4': 'q1',
 '5': 'q1q2',
'6': 'q1',
'7': 'q1',
'8': 'q1',
'9': 'q1'},
'null': {'0': 'null',
'1': 'null',
'2': 'null',
'3': 'null',
'4': 'null',
'5': 'null',
'6': 'null',
'7': 'null',
'8': 'null',
'9': 'null'}}}

In [ ]:
draw(minimize(nfa2))